In [2]:
import pymc3

In [3]:
import numpy as np

In [4]:
T = 1000
k = 5
d1 = 20
d2 = 30

In [5]:
W1 = np.random.randn(d1, k)
W2 = np.random.randn(d2, k)
mu1 = np.random.randn(d1, 1)
mu2 = np.random.randn(d2, 1)
pre_Psi1 = np.random.randn(2 * d1, d1)
Psi1 = np.dot(pre_Psi1.T, pre_Psi1)
pre_Psi2 = np.random.randn(2 * d2, d2)
Psi2 = np.dot(pre_Psi2.T, pre_Psi2)

In [6]:
Psi1_sqrt = np.linalg.cholesky(Psi1)
Psi2_sqrt = np.linalg.cholesky(Psi2)

In [7]:
Z = np.random.randn(T, k)
Z_lift1 = np.dot(W1, Z.T).T
Z_lift2 = np.dot(W2, Z.T).T
Z_shift1 = Z_lift1 + mu1.T
Z_shift2 = Z_lift2 + mu2.T
X1_noise = np.random.randn(T, d1)
X2_noise = np.random.randn(T, d2)
X1 = np.dot(X1_noise, Psi1_sqrt) + Z_shift1
X2 = np.dot(X2_noise, Psi2_sqrt) + Z_shift2

In [8]:
import pymc3 as pm

In [31]:
model = pm.Model()

with model:
    # Priors on Z-lifting matrices
    W1_var = pm.Normal('W1', shape=(d1,k))
    W2_var = pm.Normal('W2', shape=(d2,k))
    
    # Priors on X means
    mu1_var = pm.Normal('mu1', shape=(d1,1))
    mu2_var = pm.Normal('mu2', shape=(d2,1))
    
    # Priors on covariance matrices
    Psi1_var = pm.LKJCholeskyCov(
        'Psi1', n=d1, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    Psi2_var = pm.LKJCholeskyCov(
        'Psi2', n=d2, eta=2., sd_dist=pm.HalfCauchy.dist(2.5))
    
    # SDs for the Xs
    L1 = pm.expand_packed_triangular(d1, Psi1_var)
    L2 = pm.expand_packed_triangular(d2, Psi2_var)
    
    # Likelihood on Z
    Z_var = pm.Normal('Z', shape=(T,k))
    
    # Means for conditional likelihood on Xs conditioned on Z
    Z_lift1_var = W1_var.dot(Z_var.T).T
    Z_lift2_var = W2_var.dot(Z_var.T).T
    Z_shift1_var = Z_lift1_var + mu1_var.T
    Z_shift2_var = Z_lift2_var + mu2_var.T
    
    # Conditional likelihoods of Xs conditioned on Z
    X1_var = pm.MvNormal(name='X1', mu=Z_shift1_var, chol=L1, observed=X1)
    X2_var = pm.MvNormal(name='X2', mu=Z_shift2_var, chol=L2, observed=X2)

map_estimate = pm.find_MAP(model=model)

In [ ]:
map_estimate = pm.find_MAP(model=model)